In [1]:
import jettool
import pandas
import numpy
import requests
import json
#import tejapi
#tejapi.ApiConfig.api_key ="GDEy0mWAGqnI3EemCREGREZMcEVbnF"

# 初始化

In [2]:
# 初始化物件，可同時set Key
tejtool = jettool.jet.engine(api_key="6CSAoCCL7E2sgmPaSTTNGo2Y1eB0Dg")
#tejtool.load_data()
# 設定keyID，會自動查詢需要的資料表索引構造
#tejtool.set_apikey("GDEy0mWAGqnI3EemCREGREZMcEVbnF")

['使用者名稱：趙育祥_外包(ZYX)', '使用權限日期：2019-08-29/2020-12-31', '日連線次數狀態：354/100000', '日查詢資料量狀態：2106149/10000000', '月查 詢資料量狀態：3302750/9223372036854775807']


# 資料結構資訊

In [3]:
# 尋找某個table的mapping資料次屬性，可以用來知道你查的table，在其他國家是什麼table
tejtool.dbapi.get_table_mapping(market='TWN',id='AIND')

{'subId': 14,
 'subName': '基本資料',
 'description': '公司基本資料',
 'ordinal': 1,
 'tableMap': [{'tableId': 'TWN/AIND', 'dbCode': 'TWN', 'icon': 'twn.png'},
  {'tableId': 'CHN/CBAS', 'dbCode': 'CHN', 'icon': 'chn.png'},
  {'tableId': 'HKG/HIND', 'dbCode': 'HKG', 'icon': 'hkg.png'},
  {'tableId': 'JPN/JIND', 'dbCode': 'JPN', 'icon': 'jpn.png'},
  {'tableId': 'KOR/KIND', 'dbCode': 'KOR', 'icon': 'kor.png'}]}

In [4]:
# 取得該table除了PK以外的欄位名稱，用來簡化查詢步驟，無腦一口氣整個表匯入
tejtool.dbapi.get_table_columns(table_name='TWN/AFF_RAW')

['市值',
 '淨值市價比',
 '益本比',
 '股利殖利率',
 '動能(T-12~T-2)',
 '短期反轉(T-1)',
 '長期反轉(T-60~T-13)',
 '盈利能力(ROE)',
 '投資(資產成長率)',
 '違約距離(MERTON)',
 '違約距離(KMV)',
 '流動性',
 '個別風險',
 '產業集中度',
 '錯誤定價',
 '偏態係數',
 '研發密度']

In [5]:
# 用模糊字串來查詢確切的會計科目名稱
tejtool.find_account_name('常續性')

0       常續性稅後淨利
1    近四季常續性每股盈餘
Name: cname, dtype: object

In [6]:
# 使用此函式可查詢總經資料表內的名稱與代碼的對照
tejtool.compare_code_name(table_name='GLOBAL/GIDX',query_columns=['台灣加權股價指數','美國紐約綜合股價指數'])

(['SB01', 'SB21'], ['台灣加權股價指數', '美國紐約綜合股價指數'])

# 整合查詢器
### 一次查詢全部整合完成
#### 一個名稱、可到不同資料表查詢相對應的資料
#### 需正確定義名稱的精確差異

In [7]:
#data_name = ['常續性稅後淨利','收盤價(元)','報酬率-Ln','個別風險','長期反轉(T-60~T-13)','董監持股%','大股東持股(TSE)%','經理人持股%','單月營收(千元)']
#fama_name = tejtool.get_table_columns('TWN/AFF_RAW')
data_name = ['常續性稅後淨利','流通在外股數(千股)',{'id': 'APRCD', 'columns_cname':['開盤價(元)','收盤價(元)']},'報酬率-Ln','單月營收(千元)']
data_name = data_name 
data = tejtool.query_data(base_date='2019-12-31',window='3m',column_names=data_name)
#tejtool.save_data()
data[data['coid']=='2330']

資料起始日：2019-09-30
查詢財報資料
成功查詢會計家數:1668
最大財報資料日期:2018-06-01
APRCD 上市(櫃)未調整股價(日)
ASALE 上市(櫃)月營收盈餘
查詢日資料 最大資料日期:2019-12-31
上市(櫃)未調整股價(日) 重新查詢:True
上市(櫃)月營收盈餘 重新查詢:True
2019-12-30


,zdate,coid,常續性稅後淨利,報酬率-Ln,收盤價(元),流通在外股數(千股),單月營收(千元),開盤價(元),mdate
38116,2019-10-02,2330,0.0,-0.1787,279.5,25930380.0,1299155.0,280.0,2019-09-01
38117,2019-10-03,2330,0.0,-1.0791,276.5,25930380.0,1299155.0,274.0,2019-09-01
38118,2019-10-04,2330,0.0,0.0000,276.5,25930380.0,1299155.0,279.5,2019-09-01
38119,2019-10-07,2330,0.0,0.5410,278.0,25930380.0,1299155.0,279.0,2019-09-01
38120,2019-10-08,2330,0.0,3.0117,286.5,25930380.0,1299155.0,283.5,2019-09-01
...,...,...,...,...,...,...,...,...,...
38174,2019-12-25,2330,0.0,0.3008,333.0,25930380.0,107884396.0,332.0,2019-09-01
38175,2019-12-26,2330,0.0,0.0000,333.0,25930380.0,107884396.0,333.0,2019-09-01
38176,2019-12-27,2330,0.0,1.4903,338.0,25930380.0,107884396.0,335.0,2019-09-01
38177,2019-12-30,2330,0.0,-1.0409,334.5,25930380.0,107884396.0,338.0,2019-09-01


In [ ]:
# 將查詢結果保存，可以避免重複查詢
import dill
with open('tejtool.pkl','wb') as f:
    dill.dump(tejtool,f )
    # tejtool = dill.load(f) #此列為取出用

In [ ]:
data = tejtool.get_data(base_date='2019-10-16',window='3d',column_names=['常續性稅後淨利','收盤價(元)','報酬率-Ln','單月營收(千元)'])
data.head(5)

In [ ]:
# 用來取得 某個日期以前最接近的交易日
tejtool.get_zdate('2019-08-16')

In [ ]:
tejtool.make_famamacbethmodel(col_name='報酬率-Ln',check_index=['市值', '淨值市價比', '益本比', '股利殖利率', '動能(T-12~T-2)', '短期反轉(T-1)', '長期反轉(T-60~T-13)', '盈利能力(ROE)', '投資(資產成長率)', '違約距離(MERTON)', '違約距離(KMV)', '流動性', '個別風險', '產業集中度', '錯誤定價', '偏態係數', '研發密度'],window ='12m',alpha_rate = 5)

# 回測工具，與quantjet網站同一套

In [ ]:
# 定義計算式，然後使用back_test函式即可回測，目前欠缺自動查詢函式內需要資料欄位(即quantjet網站js檔的功能)的程序

def calculate():
    tejtool.data['EP'] = tejtool.data['常續性稅後淨利']/tejtool.data['收盤價(元)']
    tejtool.data['常續性稅後淨利檢查'] = tejtool.check_above('EP',0,window='1d')
def evaluate():
    tejtool.data['購入'] = tejtool.check_condition(conditions=['常續性稅後淨利檢查'])
    tejtool.data['unit'] = tejtool.equal_pv()

# 另外必須輸入用來當作市價與報酬率的欄位名稱，如:roib_name='報酬率-Ln',closed_name ='收盤價(元)'
# 若不輸入，則使用此預設值
tejtool.back_test(back_interval='2019-10-31',cash=1000000000,calculate=calculate,evaluate=evaluate)

In [ ]:
def calculate(tejtool):
    tejtool.data['報酬率']=tejtool.calculate_growthrate(check_index='收盤價(元)',window='1m',fix_date='01',method='geometric')
    tejtool.data['famamodel']=tejtool.make_famamacbethmodel(col_name='報酬率-Ln',check_index=['市值', '淨值市價比', '益本比', '股利殖利率', '動能(T-12~T-2)', '短期反轉(T-1)', '長期反轉(T-60~T-13)', '盈利能力(ROE)', '投資(資產成長率)', '違約距離(MERTON)', '違約距離(KMV)', '流動性', '個別風險', '產業集中度', '錯誤定價', '偏態係數', '研發密度'],window ='24m',alpha_rate = 5,reset_list = '01')
    tejtool.data['famascore']=tejtool.run_famascore(rank_above=100,class_count = 10) 
    tejtool.data['fama分組當月報酬']=tejtool.calculate_crossing(check_index='報酬率',window='1d',weight='市值',col_kind='mean')
def evaluate(tejtool):
    tejtool.data['fama分組判斷'] =tejtool.abnormal_selection(check_index='fama分組當月報酬',group_name='famascore',window='12m',keep='first')
    tejtool.data['購入']=tejtool.check_condition(conditions=['fama分組判斷'],check_type='and') 
    tejtool.data['unit'] = tejtool.equal_pv()
tejtool.back_test(back_interval='2018-12-31',cash=1000000000,calculate=calculate,evaluate=evaluate)

In [ ]:
tejtool.data.loc[tejtool.data['coid']=='1603',['unit','roibNext','前期持股','收盤價(元)','報酬率-Ln','手續費']]

In [ ]:
tejtool.all_date_data[(tejtool.all_date_data['coid']=='1603')&(tejtool.all_date_data['zdate']>='2019-08-15')]